# Probabilistic Model for Concept Bottleneck

This notebook demonstrates how to:
1. Load and prepare data with concept annotations
2. Define Variables and their probabilistic dependencies
3. Build a Probabilistic Model (ProbabilisticModel) with ParametricCPDs
4. Use inference engines to query the ProbabilisticModel
5. Train the model with concept and task supervision
6. Apply interventions to manipulate concept predictions in the ProbabilisticModel framework

## 1. Imports

We import the necessary libraries:
- **PyTorch**: for neural network building blocks and distributions
- **sklearn**: for evaluation metrics
- **torch_concepts**: for Variables, ParametricCPDs, ProbabilisticModel, and inference mechanisms

In [11]:
import torch
from sklearn.metrics import accuracy_score
from torch.distributions import Bernoulli, RelaxedOneHotCategorical

from torch_concepts import Annotations, AxisAnnotation, Variable
from torch_concepts.data import ToyDataset
from torch_concepts.nn import (
    LinearZC,
    LinearCC,
    ParametricCPD,
    ProbabilisticModel,
    RandomPolicy, 
    DoIntervention, 
    intervention, 
    DeterministicInference
)

## 2. Data Loading and Preparation

We load the XOR toy dataset and prepare the training data:
- **Features (x_train)**: input features for the model
- **Concepts (c_train)**: intermediate concept labels (binary: C1, C2)
- **Targets (y_train)**: task labels (converted to one-hot encoding with 2 classes)
- **Names**: concept and task attribute names

In [12]:
# Hyperparameters
latent_dims = 10
n_epochs = 500
n_samples = 1000
concept_reg = 0.5

# Load toy XOR dataset
data = ToyDataset('xor', size=n_samples, random_state=42)
x_train = data.data
c_train = data.concept_labels
y_train = data.target_labels
concept_names = data.concept_attr_names
task_names = data.task_attr_names

# Convert y_train to one-hot encoding (2 classes)
y_train = torch.cat([y_train, 1 - y_train], dim=1)

# Define concept names for the ProbabilisticModel
concept_names = ['c1', 'c2']

print(f"Dataset loaded:")
print(f"  Features shape: {x_train.shape}")
print(f"  Concepts shape: {c_train.shape}")
print(f"  Targets shape: {y_train.shape}")
print(f"  Concept names: {concept_names}")
print(f"  Task name: xor")

Dataset loaded:
  Features shape: torch.Size([1000, 2])
  Concepts shape: torch.Size([1000, 2])
  Targets shape: torch.Size([1000, 2])
  Concept names: ['c1', 'c2']
  Task name: xor


## 3. Variables: Defining the Graphical Structure

In a Probabilistic Model, **Variables** represent random variables with:
- **Name**: identifier for the variable
- **Parents**: list of parent variables (defines the graph structure)
- **Distribution**: probability distribution type (e.g., Bernoulli, Categorical)
- **Size**: dimensionality of the variable

We define:
1. **input_var (emb)**: Latent embedding with no parents (root node)
2. **concepts (c1, c2)**: Binary concepts that depend on the embedding
3. **tasks (xor)**: Categorical task output that depends on the concepts

This creates a graph: `emb → [c1, c2] → xor`

In [ ]:
# Define the latent variable (embedding)
input_var = Variable("input", parents=[], size=latent_dims)

# Define concept variables (depend on embedding)
concepts = Variable(concept_names, parents=["input"], distribution=Bernoulli)

# Define task variable (depends on concepts)
tasks = Variable("xor", parents=concept_names, distribution=RelaxedOneHotCategorical, size=2)

print("Variable structure:")
print(f"\nLatent variable:")
print(f"  Name: {input_var.concepts}")
print(f"  Parents: {input_var.parents}")
print(f"  Size: {input_var.size}")

print(f"\nConcept variables:")
for i, c in enumerate(concepts):
    print(f"  Variable {i+1}:")
    print(f"    Name: {c.concepts}")
    print(f"    Parents: {c.parents}")
    print(f"    Distribution: {c.distribution.__name__}")
    print(f"    Size: {c.size}")

print(f"\nTask variable:")
print(f"  Name: {tasks.concepts}")
print(f"  Parents: {tasks.parents}")
print(f"  Distribution: {tasks.distribution.__name__}")
print(f"  Size: {tasks.size}")

## 4. ParametricCPDs: Neural Network Components

**ParametricCPDs** are the computational units in the ProbabilisticModel that define the conditional probability distributions:
- Each ParametricCPD takes parent variables as input and produces a child variable
- ParametricCPDs are implemented as neural network modules

We define three ParametricCPDs:
1. **Backbone**: Maps input features to latent embedding (x → emb)
2. **Concept Encoder**: Maps embedding to concept endogenous (emb → [c1, c2])
3. **Task Predictor**: Maps concept endogenous to task predictions ([c1, c2] → xor)

In [ ]:
# ParametricCPD 1: Backbone (input features -> embedding)
backbone = ParametricCPD(
    "input",
    parametrization=torch.nn.Sequential(
        torch.nn.Linear(x_train.shape[1], latent_dims), 
        torch.nn.LeakyReLU()
    )
)

# ParametricCPD 2: Concept encoder (embedding -> concepts)
c_encoder = ParametricCPD(
    ["c1", "c2"], 
    parametrization=LinearZC(
        in_features=latent_dims,
        out_features=concepts[0].size
    )
)

# ParametricCPD 3: Task predictor (concepts -> task)
y_predictor = ParametricCPD(
    "xor", 
    parametrization=LinearCC(
        in_features_endogenous=sum(c.size for c in concepts),
        out_features=tasks.size
    )
)

print("ParametricCPD structure:")
print(f"\n1. Backbone ParametricCPD:")
print(f"   Variable: emb")
print(f"   Input size: {x_train.shape[1]}")
print(f"   Output size: {latent_dims}")

print(f"\n2. Concept Encoder ParametricCPD:")
print(f"   Variables: {['c1', 'c2']}")
print(f"   Input: embedding of size {latent_dims}")
print(f"   Output: concept endogenous of size {concepts[0].size}")

print(f"\n3. Task Predictor ParametricCPD:")
print(f"   Variable: xor")
print(f"   Input: concept endogenous of size {sum(c.size for c in concepts)}")
print(f"   Output: task endogenous of size {tasks.size}")

## 5. Probabilistic Model (ProbabilisticModel)

The **ProbabilisticModel** combines Variables and ParametricCPDs into a coherent model:
- It represents the joint probability distribution over all variables
- It manages the computational graph defined by parent-child relationships
- It provides an interface for inference and learning

The ProbabilisticModel encapsulates:
- All variables: latent, concepts, and tasks
- All CPDs: backbone, concept encoder, and task predictor

In [15]:
# Initialize the Probabilistic Model
concept_model = ProbabilisticModel(
    variables=[input_var, *concepts, tasks],
    parametric_cpds=[backbone, *c_encoder, y_predictor]
)

print("Probabilistic Model:")
print(concept_model)
print(f"\nNumber of variables: {len(concept_model.variables)}")
print(f"Variable names: {[v.concepts for v in concept_model.variables]}")
print(f"\nNumber of CPDs: {len(concept_model.parametric_cpds)}")
print(f"\nGraph structure:")
print(f"  emb (latent) → [c1, c2] (concepts) → xor (task)")

Probabilistic Graphical Model:
ProbabilisticGraphicalModel(
  (factors): ModuleDict(
    (emb): Factor(concepts=['emb'], module=Sequential)
    (c1): Factor(concepts=['c1'], module=ProbEncoderFromEmb)
    (c2): Factor(concepts=['c2'], module=ProbEncoderFromEmb)
    (xor): Factor(concepts=['xor'], module=ProbPredictor)
  )
)

Number of variables: 4
Variable names: [['emb'], ['c1'], ['c2'], ['xor']]

Number of factors: 4

Graph structure:
  emb (latent) → [c1, c2] (concepts) → xor (task)


## 6. Inference Engine

The **DeterministicInference** engine performs inference on the ProbabilisticModel:
- **Evidence**: Known/observed variables (e.g., input features)
- **Query**: Variables we want to predict
- **Inference**: Forward pass through the graph to compute query variables

We set up:
- **Initial input**: The embedding variable (computed from x_train)
- **Query concepts**: We want to infer c1, c2, and xor

In [16]:
# Initialize the inference engine
inference_engine = DeterministicInference(concept_model)

# Define the evidence (what we observe)
initial_input = {'emb': x_train}

# Define the query (what we want to infer)
query_concepts = ["c1", "c2", "xor"]

print("Inference setup:")
print(f"  Engine: DeterministicInference")
print(f"  Evidence variable: emb (from input features)")
print(f"  Query variables: {query_concepts}")
print(f"\nInference will compute: x_train → emb → [c1, c2] → xor")

Inference setup:
  Engine: DeterministicInference
  Evidence variable: emb (from input features)
  Query variables: ['c1', 'c2', 'xor']

Inference will compute: x_train → emb → [c1, c2] → xor


## 7. Training

We train the ProbabilisticModel with a combined loss:
- **Concept loss**: BCE loss between predicted and true concept labels (c1, c2)
- **Task loss**: BCE loss between predicted and true task labels (xor)
- **Total loss**: `concept_loss + concept_reg * task_loss`

During training:
1. Query the inference engine to get predictions for c1, c2, and xor
2. Split the output into concept and task predictions
3. Compute losses and backpropagate through the entire ProbabilisticModel

In [17]:
# Setup training
optimizer = torch.optim.AdamW(concept_model.parameters(), lr=0.01)
loss_fn = torch.nn.BCEWithLogitsLoss()
concept_model.train()

# Training loop
for epoch in range(n_epochs):
    optimizer.zero_grad()

    # Inference: query the ProbabilisticModel for concept and task predictions
    cy_pred = inference_engine.query(query_concepts, evidence=initial_input)
    
    # Split predictions: first columns are concepts, remaining are task
    c_pred = cy_pred[:, :c_train.shape[1]]
    y_pred = cy_pred[:, c_train.shape[1]:]

    # Compute loss
    concept_loss = loss_fn(c_pred, c_train)
    task_loss = loss_fn(y_pred, y_train)
    loss = concept_loss + concept_reg * task_loss

    # Backward pass
    loss.backward()
    optimizer.step()

    # Log progress
    if epoch % 100 == 0:
        task_accuracy = accuracy_score(y_train, y_pred > 0.)
        concept_accuracy = accuracy_score(c_train, c_pred > 0.)
        print(f"Epoch {epoch}: Loss {loss.item():.2f} | Task Acc: {task_accuracy:.2f} | Concept Acc: {concept_accuracy:.2f}")

print("\nTraining complete!")

Epoch 0: Loss 1.05 | Task Acc: 0.49 | Concept Acc: 0.25
Epoch 100: Loss 0.51 | Task Acc: 0.02 | Concept Acc: 0.97
Epoch 200: Loss 0.43 | Task Acc: 0.29 | Concept Acc: 0.98
Epoch 300: Loss 0.41 | Task Acc: 0.31 | Concept Acc: 0.99
Epoch 400: Loss 0.39 | Task Acc: 0.32 | Concept Acc: 0.99

Training complete!


## 8. Baseline Predictions (No Intervention)

Let's examine the model's predictions without any interventions.
The output contains concatenated predictions: [c1, c2, xor]

In [18]:
# Get baseline predictions
concept_model.eval()
with torch.no_grad():
    cy_pred = inference_engine.query(query_concepts, evidence=initial_input)

print("Baseline predictions (first 5 samples):")
print("Format: [c1, c2, xor_class0, xor_class1]")
print(cy_pred[:5])
print(f"\nShape: {cy_pred.shape}")
print(f"  Columns 0-1: concept predictions (c1, c2)")
print(f"  Columns 2-3: task predictions (xor one-hot)")

Baseline predictions (first 5 samples):
Format: [c1, c2, xor_class0, xor_class1]
tensor([[-3.8935e+00,  1.8834e+01,  1.0420e-01, -1.0441e-01],
        [ 8.8618e+00,  4.0058e+00, -4.0338e-03,  5.3845e-03],
        [-1.1902e+01, -1.3458e+01,  3.8285e-02, -4.0555e-02],
        [-1.3823e+01,  1.3051e+01,  1.0638e-01, -1.0663e-01],
        [ 4.0281e+00,  8.7874e+00, -9.3093e-04,  2.2892e-03]])

Shape: torch.Size([1000, 4])
  Columns 0-1: concept predictions (c1, c2)
  Columns 2-3: task predictions (xor one-hot)


## 9. Interventions in ProbabilisticModel

Interventions in the ProbabilisticModel framework work as follows:
- We can set (do-operation) specific concept values
- The effects propagate through the graph to downstream variables

### Intervention Setup:
- **Policy**: RandomPolicy to randomly select samples and intervene on concept c1
- **Strategy**: DoIntervention to set c1 to a constant value (-10)
- **Layer**: Intervene at the "c1.encoder" CPD
- **Quantile**: 1.0 (intervene on all selected samples)

In [19]:
# Create annotations for intervention
int_policy_c = RandomPolicy(out_features=concept_model.concept_to_variable["c1"].size, scale=100)
int_strategy_c = DoIntervention(model=concept_model.parametric_cpds, constants=-10)

print("Intervention configuration:")
print(f"  Policy: RandomPolicy on concept 'c1'")
print(f"  Strategy: DoIntervention with constant value -10")
print(f"  Target layer: c1.encoder")
print(f"  Quantile: 1.0 (intervene on all selected samples)")
print(f"\nThis intervention will:")
print(f"  1. Randomly select samples")
print(f"  2. Set concept c1 to -10 for those samples")
print(f"  3. Propagate the effect to the task prediction (xor)")

Intervention configuration:
  Policy: RandomPolicy on concept 'c1'
  Strategy: DoIntervention with constant value -10
  Target layer: c1.encoder
  Quantile: 1.0 (intervene on all selected samples)

This intervention will:
  1. Randomly select samples
  2. Set concept c1 to -10 for those samples
  3. Propagate the effect to the task prediction (xor)


## 10. Applying the Intervention

Now we apply the intervention and observe how the predictions change.
Compare these results with the baseline predictions above to see the intervention's effect.

In [20]:
print("Predictions with intervention:")
with intervention(policies=int_policy_c,
                  strategies=int_strategy_c,
                  target_concepts=["c1", "c2"]):
    cy_pred_intervened = inference_engine.query(query_concepts, evidence=initial_input)
    print("Format: [c1, c2, xor_class0, xor_class1]")
    print(cy_pred_intervened[:5])

print("\nNote: Compare with baseline predictions above.")
print("You should see c1 values changed to -10 for randomly selected samples,")
print("and corresponding changes in the xor predictions.")

Predictions with intervention:
Format: [c1, c2, xor_class0, xor_class1]
tensor([[-10.0000, -10.0000,   0.0383,  -0.0406],
        [-10.0000, -10.0000,   0.0383,  -0.0406],
        [-10.0000, -10.0000,   0.0383,  -0.0406],
        [-10.0000, -10.0000,   0.0383,  -0.0406],
        [-10.0000, -10.0000,   0.0383,  -0.0406]], grad_fn=<SliceBackward0>)

Note: Compare with baseline predictions above.
You should see c1 values changed to -10 for randomly selected samples,
and corresponding changes in the xor predictions.


## Summary

In this notebook, we explored Probabilistic Models for concept-based learning:

1. **Data**: Loaded the XOR toy dataset with binary concepts
2. **Variables**: Defined the graphical structure with latent, concept, and task variables
3. **ParametricCPDs**: Created neural network components that compute conditional probabilities
4. **ProbabilisticModel**: Combined variables and CPDs into a coherent probabilistic model
5. **Inference**: Used deterministic inference to query the model
6. **Training**: Trained with combined concept and task supervision
7. **Interventions**: Applied causal interventions to manipulate concepts and observe effects

### Key Advantages of ProbabilisticModel Framework:
- **Explicit graph structure**: Clear representation of variable dependencies
- **Probabilistic reasoning**: Each variable has an associated distribution
- **Causal interventions**: Do-calculus operations for counterfactual analysis
- **Modularity**: Easy to add/remove variables and CPDs
- **Interpretability**: Graph structure makes the model's reasoning transparent

This framework is particularly powerful for:
- Causal reasoning and counterfactual analysis
- Models with complex variable dependencies
- Scenarios requiring explicit probabilistic modeling
- Interpretable AI applications